In [1]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

120
500
1000


# GCN using only domain Feature

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [5]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 50, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0683, Train: 52.50%, Valid: 25.40% Test: 23.00%
Run: 01, Epoch: 02, Loss: 1.1706, Train: 81.67%, Valid: 31.20% Test: 29.50%
Run: 01, Epoch: 03, Loss: 0.8812, Train: 92.50%, Valid: 37.20% Test: 35.30%
Run: 01, Epoch: 04, Loss: 0.6316, Train: 96.67%, Valid: 41.20% Test: 39.80%
Run: 01, Epoch: 05, Loss: 0.5482, Train: 98.33%, Valid: 43.20% Test: 42.20%
Run: 01, Epoch: 06, Loss: 0.4717, Train: 98.33%, Valid: 44.20% Test: 43.90%
Run: 01, Epoch: 07, Loss: 0.4040, Train: 98.33%, Valid: 45.00% Test: 45.40%
Run: 01, Epoch: 08, Loss: 0.3327, Train: 100.00%, Valid: 47.40% Test: 46.70%
Run: 01, Epoch: 09, Loss: 0.2505, Train: 100.00%, Valid: 47.80% Test: 47.50%
Run: 01, Epoch: 10, Loss: 0.1930, Train: 100.00%, Valid: 48.20% Test: 48.40%
Run: 01, Epoch: 11, Loss: 0.1781, Train: 100.00%, Valid: 48.80% Test: 48.10%
Run: 01, Epoch: 12, Loss: 0.1812, Train: 100.00%, Valid: 48.60% Test: 47.80%
Run: 01, Epoch: 13, Loss: 0.1480, Train: 100.00%, Valid: 48.20% Test: 48.20%
Run: 0

Run: 03, Epoch: 06, Loss: 0.4149, Train: 99.17%, Valid: 44.40% Test: 44.50%
Run: 03, Epoch: 07, Loss: 0.3757, Train: 99.17%, Valid: 44.60% Test: 44.40%
Run: 03, Epoch: 08, Loss: 0.4059, Train: 100.00%, Valid: 44.60% Test: 43.90%
Run: 03, Epoch: 09, Loss: 0.2728, Train: 100.00%, Valid: 45.20% Test: 44.00%
Run: 03, Epoch: 10, Loss: 0.2829, Train: 100.00%, Valid: 44.00% Test: 43.20%
Run: 03, Epoch: 11, Loss: 0.2453, Train: 100.00%, Valid: 44.40% Test: 43.50%
Run: 03, Epoch: 12, Loss: 0.1530, Train: 100.00%, Valid: 44.80% Test: 43.30%
Run: 03, Epoch: 13, Loss: 0.1561, Train: 100.00%, Valid: 44.00% Test: 43.00%
Run: 03, Epoch: 14, Loss: 0.1364, Train: 100.00%, Valid: 44.00% Test: 42.60%
Run: 03, Epoch: 15, Loss: 0.1459, Train: 100.00%, Valid: 44.40% Test: 42.50%
Run: 03, Epoch: 16, Loss: 0.1342, Train: 100.00%, Valid: 43.60% Test: 42.10%
Run: 03, Epoch: 17, Loss: 0.1587, Train: 100.00%, Valid: 44.20% Test: 42.70%
Run: 03, Epoch: 18, Loss: 0.1205, Train: 100.00%, Valid: 44.00% Test: 43.50%
R

Run: 05, Epoch: 12, Loss: 0.1960, Train: 100.00%, Valid: 53.00% Test: 47.50%
Run: 05, Epoch: 13, Loss: 0.1634, Train: 100.00%, Valid: 52.80% Test: 47.60%
Run: 05, Epoch: 14, Loss: 0.1837, Train: 100.00%, Valid: 52.20% Test: 47.80%
Run: 05, Epoch: 15, Loss: 0.1387, Train: 100.00%, Valid: 52.00% Test: 48.30%
Run: 05, Epoch: 16, Loss: 0.0892, Train: 100.00%, Valid: 52.00% Test: 48.60%
Run: 05, Epoch: 17, Loss: 0.1734, Train: 100.00%, Valid: 51.20% Test: 49.00%
Run: 05, Epoch: 18, Loss: 0.1339, Train: 100.00%, Valid: 51.20% Test: 48.70%
Run: 05, Epoch: 19, Loss: 0.1084, Train: 100.00%, Valid: 50.80% Test: 49.10%
Run: 05, Epoch: 20, Loss: 0.0940, Train: 100.00%, Valid: 51.00% Test: 49.40%
Run: 05, Epoch: 21, Loss: 0.0934, Train: 100.00%, Valid: 51.20% Test: 49.60%
Run: 05, Epoch: 22, Loss: 0.0904, Train: 100.00%, Valid: 51.40% Test: 49.90%
Run: 05, Epoch: 23, Loss: 0.0799, Train: 100.00%, Valid: 51.80% Test: 50.00%
Run: 05, Epoch: 24, Loss: 0.1523, Train: 100.00%, Valid: 51.40% Test: 49.90%

Run: 07, Epoch: 18, Loss: 0.0998, Train: 100.00%, Valid: 54.60% Test: 53.70%
Run: 07, Epoch: 19, Loss: 0.0459, Train: 100.00%, Valid: 54.60% Test: 53.50%
Run: 07, Epoch: 20, Loss: 0.0660, Train: 100.00%, Valid: 54.60% Test: 53.10%
Run: 07, Epoch: 21, Loss: 0.0972, Train: 100.00%, Valid: 54.40% Test: 53.10%
Run: 07, Epoch: 22, Loss: 0.0715, Train: 100.00%, Valid: 53.80% Test: 53.00%
Run: 07, Epoch: 23, Loss: 0.0632, Train: 100.00%, Valid: 53.80% Test: 52.90%
Run: 07, Epoch: 24, Loss: 0.0867, Train: 100.00%, Valid: 54.00% Test: 52.70%
Run: 07, Epoch: 25, Loss: 0.0549, Train: 100.00%, Valid: 53.80% Test: 52.80%
Run: 07, Epoch: 26, Loss: 0.0356, Train: 100.00%, Valid: 53.40% Test: 52.50%
Run: 07, Epoch: 27, Loss: 0.0499, Train: 100.00%, Valid: 53.40% Test: 52.70%
Run: 07, Epoch: 28, Loss: 0.0523, Train: 100.00%, Valid: 53.00% Test: 52.60%
Run: 07, Epoch: 29, Loss: 0.0545, Train: 100.00%, Valid: 52.80% Test: 52.80%
Run: 07, Epoch: 30, Loss: 0.0661, Train: 100.00%, Valid: 52.80% Test: 52.60%

Run: 09, Epoch: 24, Loss: 0.0932, Train: 100.00%, Valid: 48.00% Test: 49.30%
Run: 09, Epoch: 25, Loss: 0.0994, Train: 100.00%, Valid: 48.20% Test: 48.90%
Run: 09, Epoch: 26, Loss: 0.1231, Train: 100.00%, Valid: 48.20% Test: 48.80%
Run: 09, Epoch: 27, Loss: 0.0842, Train: 100.00%, Valid: 47.80% Test: 49.00%
Run: 09, Epoch: 28, Loss: 0.1117, Train: 100.00%, Valid: 47.80% Test: 48.70%
Run: 09, Epoch: 29, Loss: 0.0929, Train: 100.00%, Valid: 47.80% Test: 48.40%
Run: 09, Epoch: 30, Loss: 0.0472, Train: 100.00%, Valid: 48.00% Test: 48.40%
Run: 09, Epoch: 31, Loss: 0.0705, Train: 100.00%, Valid: 48.20% Test: 48.70%
Run: 09, Epoch: 32, Loss: 0.0514, Train: 100.00%, Valid: 48.60% Test: 48.50%
Run: 09, Epoch: 33, Loss: 0.0486, Train: 100.00%, Valid: 48.60% Test: 48.80%
Run: 09, Epoch: 34, Loss: 0.0525, Train: 100.00%, Valid: 48.80% Test: 49.00%
Run: 09, Epoch: 35, Loss: 0.0456, Train: 100.00%, Valid: 48.80% Test: 49.00%
Run: 09, Epoch: 36, Loss: 0.0294, Train: 100.00%, Valid: 49.00% Test: 48.90%

# Create Class Contrast feature

In [8]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [4]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,3694,3695,3696,3697,3698,3699,3700,3701,3702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [5]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [11]:

Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    #print(f)
    f.clear()
    Fec.append(vec)

In [12]:
print(Fec)

[[27, 28, 28, 31, 26, 29], [26, 33, 31, 28, 30, 28], [24, 24, 24, 24, 24, 25], [34, 32, 37, 36, 38, 44], [32, 34, 33, 36, 34, 34], [27, 34, 31, 30, 30, 25], [32, 36, 34, 37, 30, 32], [35, 34, 31, 31, 33, 29], [23, 25, 27, 27, 27, 27], [33, 35, 34, 34, 35, 36], [23, 30, 31, 25, 26, 27], [30, 30, 35, 34, 36, 32], [25, 37, 37, 31, 28, 27], [32, 36, 36, 32, 31, 28], [16, 18, 20, 20, 17, 18], [30, 35, 36, 37, 32, 32], [21, 24, 33, 28, 27, 26], [29, 33, 36, 33, 37, 29], [20, 24, 25, 25, 27, 27], [26, 26, 25, 25, 22, 25], [32, 35, 34, 32, 32, 32], [28, 30, 27, 29, 27, 32], [23, 26, 27, 25, 27, 31], [27, 28, 29, 29, 25, 29], [23, 21, 23, 22, 22, 25], [24, 29, 29, 25, 24, 24], [32, 32, 35, 34, 35, 37], [20, 29, 31, 22, 21, 21], [25, 25, 30, 28, 31, 29], [33, 36, 38, 35, 36, 38], [26, 27, 28, 26, 26, 27], [15, 16, 18, 17, 18, 18], [22, 23, 24, 20, 27, 20], [31, 34, 32, 33, 31, 34], [28, 33, 33, 31, 30, 31], [25, 29, 26, 30, 26, 25], [21, 24, 24, 26, 26, 24], [24, 23, 25, 24, 25, 26], [20, 21, 22

In [13]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0

    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [14]:
print(SFec)

[[7, 7, 10, 6, 6, 8], [4, 6, 4, 3, 4, 5], [6, 4, 5, 5, 10, 10], [3, 2, 4, 5, 4, 7], [5, 7, 5, 7, 8, 6], [4, 6, 5, 5, 4, 4], [5, 6, 8, 5, 3, 3], [10, 10, 8, 5, 5, 6], [3, 2, 2, 1, 5, 1], [3, 5, 4, 4, 5, 3], [4, 4, 8, 4, 4, 2], [5, 3, 2, 3, 5, 5], [5, 6, 6, 2, 3, 2], [6, 11, 8, 7, 7, 2], [3, 2, 2, 2, 0, 0], [2, 5, 7, 6, 4, 3], [1, 2, 9, 3, 2, 1], [4, 5, 5, 5, 7, 4], [6, 4, 3, 5, 6, 4], [3, 4, 4, 3, 2, 1], [7, 9, 5, 4, 4, 3], [4, 4, 5, 5, 5, 9], [3, 2, 3, 2, 2, 6], [2, 4, 6, 5, 2, 3], [2, 2, 3, 6, 6, 9], [2, 2, 4, 3, 1, 1], [8, 7, 4, 7, 8, 8], [1, 4, 7, 3, 3, 1], [4, 4, 2, 5, 6, 4], [4, 4, 12, 6, 3, 3], [3, 5, 6, 4, 2, 3], [4, 4, 7, 4, 4, 3], [3, 4, 3, 2, 5, 2], [6, 7, 9, 8, 7, 7], [5, 5, 7, 6, 11, 8], [2, 4, 5, 5, 3, 3], [4, 3, 3, 3, 5, 3], [2, 2, 1, 2, 3, 4], [6, 8, 8, 10, 5, 5], [1, 2, 5, 5, 3, 3], [5, 5, 9, 7, 6, 5], [4, 5, 3, 1, 3, 2], [2, 3, 7, 7, 4, 5], [4, 4, 6, 2, 4, 0], [3, 4, 4, 3, 5, 4], [11, 13, 8, 8, 6, 7], [1, 2, 3, 2, 3, 2], [4, 3, 2, 3, 4, 3], [1, 2, 3, 3, 2, 1], [0, 2, 6

In [15]:
Tdata = pd.DataFrame(Fec)
T1data = pd.DataFrame(SFec)
#data.insert(loc=k,column='Class',value=Node_Class)

result = pd.concat([Tdata, T1data], axis=1)
result.head()
compression_opts = dict(method='zip',
                        archive_name='CC_Feature_Citeseer.csv')
result.to_csv('CC_Feature_Citeseer.zip', index=True,
          compression=compression_opts)

In [16]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[27., 28., 28.,  ...,  6.,  6.,  8.],
        [26., 33., 31.,  ...,  3.,  4.,  5.],
        [24., 24., 24.,  ...,  5., 10., 10.],
        ...,
        [35., 37., 38.,  ...,  6.,  3.,  4.],
        [33., 36., 34.,  ...,  2.,  3.,  1.],
        [22., 23., 22.,  ...,  2.,  3.,  6.]])


'torch.FloatTensor'

# GCN using CC feature

In [17]:
data.x=CC_domain
print(data)

Data(x=[3327, 12], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [19]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0662, Train: 16.67%, Valid: 20.60% Test: 22.30%
Run: 01, Epoch: 02, Loss: 1.9296, Train: 16.67%, Valid: 20.60% Test: 22.20%
Run: 01, Epoch: 03, Loss: 1.8435, Train: 16.67%, Valid: 20.60% Test: 22.10%
Run: 01, Epoch: 04, Loss: 1.7487, Train: 18.33%, Valid: 18.20% Test: 20.40%
Run: 01, Epoch: 05, Loss: 1.7605, Train: 21.67%, Valid: 11.80% Test: 13.20%
Run: 01, Epoch: 06, Loss: 1.7231, Train: 22.50%, Valid: 8.60% Test: 9.60%
Run: 01, Epoch: 07, Loss: 1.6680, Train: 27.50%, Valid: 11.00% Test: 13.70%
Run: 01, Epoch: 08, Loss: 1.5929, Train: 37.50%, Valid: 18.40% Test: 19.60%
Run: 01, Epoch: 09, Loss: 1.5225, Train: 30.83%, Valid: 20.40% Test: 21.30%
Run: 01, Epoch: 10, Loss: 1.4774, Train: 26.67%, Valid: 18.00% Test: 20.60%
Run: 01, Epoch: 11, Loss: 1.4659, Train: 30.83%, Valid: 21.00% Test: 22.80%
Run: 01, Epoch: 12, Loss: 1.3690, Train: 30.00%, Valid: 23.20% Test: 25.20%
Run: 01, Epoch: 13, Loss: 1.3146, Train: 39.17%, Valid: 32.20% Test: 30.00%
Run: 01, Epoch

Run: 01, Epoch: 124, Loss: 0.3172, Train: 79.17%, Valid: 70.60% Test: 67.60%
Run: 01, Epoch: 125, Loss: 0.2574, Train: 80.00%, Valid: 68.40% Test: 67.20%
Run: 01, Epoch: 126, Loss: 0.3180, Train: 82.50%, Valid: 70.60% Test: 69.20%
Run: 01, Epoch: 127, Loss: 0.2802, Train: 80.83%, Valid: 69.40% Test: 69.00%
Run: 01, Epoch: 128, Loss: 0.3091, Train: 80.83%, Valid: 67.60% Test: 70.10%
Run: 01, Epoch: 129, Loss: 0.2201, Train: 80.00%, Valid: 64.80% Test: 69.50%
Run: 01, Epoch: 130, Loss: 0.2457, Train: 80.00%, Valid: 60.40% Test: 66.60%
Run: 01, Epoch: 131, Loss: 0.2432, Train: 70.83%, Valid: 57.80% Test: 63.80%
Run: 01, Epoch: 132, Loss: 0.2853, Train: 75.83%, Valid: 61.60% Test: 65.00%
Run: 01, Epoch: 133, Loss: 0.2872, Train: 77.50%, Valid: 66.40% Test: 68.40%
Run: 01, Epoch: 134, Loss: 0.3030, Train: 82.50%, Valid: 69.40% Test: 69.30%
Run: 01, Epoch: 135, Loss: 0.3255, Train: 82.50%, Valid: 70.00% Test: 69.80%
Run: 01, Epoch: 136, Loss: 0.3503, Train: 85.83%, Valid: 70.80% Test: 69.70%

Run: 02, Epoch: 57, Loss: 0.5099, Train: 77.50%, Valid: 64.00% Test: 68.70%
Run: 02, Epoch: 58, Loss: 0.5645, Train: 66.67%, Valid: 58.00% Test: 61.80%
Run: 02, Epoch: 59, Loss: 0.4682, Train: 56.67%, Valid: 53.00% Test: 55.60%
Run: 02, Epoch: 60, Loss: 0.5548, Train: 55.00%, Valid: 48.60% Test: 52.60%
Run: 02, Epoch: 61, Loss: 0.5217, Train: 64.17%, Valid: 57.80% Test: 61.00%
Run: 02, Epoch: 62, Loss: 0.5215, Train: 70.00%, Valid: 65.80% Test: 66.40%
Run: 02, Epoch: 63, Loss: 0.4691, Train: 70.00%, Valid: 68.00% Test: 62.30%
Run: 02, Epoch: 64, Loss: 0.5084, Train: 75.00%, Valid: 71.00% Test: 67.20%
Run: 02, Epoch: 65, Loss: 0.5094, Train: 83.33%, Valid: 75.80% Test: 73.50%
Run: 02, Epoch: 66, Loss: 0.5901, Train: 87.50%, Valid: 75.80% Test: 74.20%
Run: 02, Epoch: 67, Loss: 0.5191, Train: 80.83%, Valid: 67.20% Test: 68.40%
Run: 02, Epoch: 68, Loss: 0.5239, Train: 67.50%, Valid: 54.40% Test: 56.90%
Run: 02, Epoch: 69, Loss: 0.4409, Train: 65.83%, Valid: 52.20% Test: 57.00%
Run: 02, Epo

Run: 02, Epoch: 181, Loss: 0.2534, Train: 83.33%, Valid: 70.60% Test: 69.90%
Run: 02, Epoch: 182, Loss: 0.2579, Train: 82.50%, Valid: 71.80% Test: 69.90%
Run: 02, Epoch: 183, Loss: 0.2376, Train: 82.50%, Valid: 71.80% Test: 68.00%
Run: 02, Epoch: 184, Loss: 0.3042, Train: 80.00%, Valid: 70.60% Test: 64.00%
Run: 02, Epoch: 185, Loss: 0.2714, Train: 79.17%, Valid: 71.60% Test: 67.20%
Run: 02, Epoch: 186, Loss: 0.2713, Train: 87.50%, Valid: 75.00% Test: 73.80%
Run: 02, Epoch: 187, Loss: 0.2358, Train: 95.83%, Valid: 74.60% Test: 76.10%
Run: 02, Epoch: 188, Loss: 0.2827, Train: 94.17%, Valid: 70.40% Test: 72.00%
Run: 02, Epoch: 189, Loss: 0.1824, Train: 84.17%, Valid: 64.20% Test: 64.00%
Run: 02, Epoch: 190, Loss: 0.2104, Train: 75.00%, Valid: 57.20% Test: 56.10%
Run: 02, Epoch: 191, Loss: 0.2775, Train: 70.00%, Valid: 55.20% Test: 54.10%
Run: 02, Epoch: 192, Loss: 0.2295, Train: 75.83%, Valid: 58.20% Test: 58.70%
Run: 02, Epoch: 193, Loss: 0.2257, Train: 80.83%, Valid: 59.80% Test: 62.80%

Run: 03, Epoch: 106, Loss: 0.3415, Train: 95.00%, Valid: 76.40% Test: 78.20%
Run: 03, Epoch: 107, Loss: 0.3448, Train: 92.50%, Valid: 74.00% Test: 77.30%
Run: 03, Epoch: 108, Loss: 0.4672, Train: 90.83%, Valid: 72.80% Test: 76.60%
Run: 03, Epoch: 109, Loss: 0.4443, Train: 87.50%, Valid: 72.00% Test: 77.00%
Run: 03, Epoch: 110, Loss: 0.2593, Train: 93.33%, Valid: 74.00% Test: 77.00%
Run: 03, Epoch: 111, Loss: 0.3963, Train: 96.67%, Valid: 76.20% Test: 76.90%
Run: 03, Epoch: 112, Loss: 0.3438, Train: 87.50%, Valid: 71.20% Test: 69.30%
Run: 03, Epoch: 113, Loss: 0.3085, Train: 75.00%, Valid: 65.80% Test: 61.70%
Run: 03, Epoch: 114, Loss: 0.3273, Train: 68.33%, Valid: 59.40% Test: 57.10%
Run: 03, Epoch: 115, Loss: 0.3251, Train: 68.33%, Valid: 60.80% Test: 58.50%
Run: 03, Epoch: 116, Loss: 0.4441, Train: 82.50%, Valid: 69.60% Test: 65.90%
Run: 03, Epoch: 117, Loss: 0.2806, Train: 86.67%, Valid: 70.40% Test: 69.60%
Run: 03, Epoch: 118, Loss: 0.3040, Train: 86.67%, Valid: 67.00% Test: 69.90%

...

Run: 08, Epoch: 06, Loss: 1.6326, Train: 35.00%, Valid: 31.00% Test: 29.20%
Run: 08, Epoch: 07, Loss: 1.5468, Train: 35.00%, Valid: 34.20% Test: 33.30%
Run: 08, Epoch: 08, Loss: 1.5396, Train: 29.17%, Valid: 30.80% Test: 30.10%
Run: 08, Epoch: 09, Loss: 1.4687, Train: 23.33%, Valid: 23.00% Test: 22.00%
Run: 08, Epoch: 10, Loss: 1.4607, Train: 20.83%, Valid: 20.60% Test: 19.10%
Run: 08, Epoch: 11, Loss: 1.3466, Train: 16.67%, Valid: 20.00% Test: 18.00%
Run: 08, Epoch: 12, Loss: 1.3404, Train: 16.67%, Valid: 20.80% Test: 18.40%
Run: 08, Epoch: 13, Loss: 1.2852, Train: 23.33%, Valid: 25.40% Test: 23.00%
Run: 08, Epoch: 14, Loss: 1.3231, Train: 30.83%, Valid: 33.40% Test: 28.80%
Run: 08, Epoch: 15, Loss: 1.2214, Train: 30.83%, Valid: 36.40% Test: 33.60%
Run: 08, Epoch: 16, Loss: 1.1505, Train: 34.17%, Valid: 42.40% Test: 37.20%
Run: 08, Epoch: 17, Loss: 1.1197, Train: 42.50%, Valid: 45.60% Test: 41.90%
Run: 08, Epoch: 18, Loss: 1.1112, Train: 49.17%, Valid: 49.80% Test: 45.80%
Run: 08, Epo

Run: 08, Epoch: 134, Loss: 0.1810, Train: 90.00%, Valid: 74.20% Test: 71.90%
Run: 08, Epoch: 135, Loss: 0.2597, Train: 92.50%, Valid: 76.00% Test: 71.90%
Run: 08, Epoch: 136, Loss: 0.2299, Train: 90.83%, Valid: 74.80% Test: 71.20%
Run: 08, Epoch: 137, Loss: 0.2440, Train: 87.50%, Valid: 74.00% Test: 71.40%
Run: 08, Epoch: 138, Loss: 0.2519, Train: 89.17%, Valid: 75.80% Test: 72.50%
Run: 08, Epoch: 139, Loss: 0.2422, Train: 90.83%, Valid: 72.00% Test: 70.30%
Run: 08, Epoch: 140, Loss: 0.2369, Train: 89.17%, Valid: 70.40% Test: 71.40%
Run: 08, Epoch: 141, Loss: 0.1995, Train: 90.83%, Valid: 71.20% Test: 71.90%
Run: 08, Epoch: 142, Loss: 0.2022, Train: 90.00%, Valid: 71.00% Test: 73.80%
Run: 08, Epoch: 143, Loss: 0.2041, Train: 88.33%, Valid: 71.60% Test: 73.60%
Run: 08, Epoch: 144, Loss: 0.2303, Train: 86.67%, Valid: 71.60% Test: 73.70%
Run: 08, Epoch: 145, Loss: 0.2497, Train: 84.17%, Valid: 72.60% Test: 73.80%
Run: 08, Epoch: 146, Loss: 0.1810, Train: 89.17%, Valid: 71.80% Test: 75.20%

Run: 09, Epoch: 63, Loss: 0.4816, Train: 73.33%, Valid: 63.80% Test: 61.20%
Run: 09, Epoch: 64, Loss: 0.4537, Train: 70.83%, Valid: 59.60% Test: 58.20%
Run: 09, Epoch: 65, Loss: 0.5130, Train: 70.83%, Valid: 60.60% Test: 57.30%
Run: 09, Epoch: 66, Loss: 0.4552, Train: 73.33%, Valid: 64.80% Test: 58.10%
Run: 09, Epoch: 67, Loss: 0.4098, Train: 75.00%, Valid: 67.20% Test: 63.10%
Run: 09, Epoch: 68, Loss: 0.4083, Train: 80.00%, Valid: 70.80% Test: 69.60%
Run: 09, Epoch: 69, Loss: 0.5282, Train: 83.33%, Valid: 75.00% Test: 73.20%
Run: 09, Epoch: 70, Loss: 0.4579, Train: 80.83%, Valid: 76.00% Test: 74.50%
Run: 09, Epoch: 71, Loss: 0.4315, Train: 75.83%, Valid: 72.20% Test: 72.50%
Run: 09, Epoch: 72, Loss: 0.4376, Train: 79.17%, Valid: 69.00% Test: 71.10%
Run: 09, Epoch: 73, Loss: 0.3887, Train: 75.00%, Valid: 60.00% Test: 64.80%
Run: 09, Epoch: 74, Loss: 0.4278, Train: 70.00%, Valid: 56.80% Test: 62.70%
Run: 09, Epoch: 75, Loss: 0.4879, Train: 69.17%, Valid: 57.20% Test: 61.30%
Run: 09, Epo

Run: 09, Epoch: 197, Loss: 0.2147, Train: 75.83%, Valid: 61.80% Test: 61.60%
Run: 09, Epoch: 198, Loss: 0.2601, Train: 71.67%, Valid: 56.00% Test: 58.70%
Run: 09, Epoch: 199, Loss: 0.2022, Train: 73.33%, Valid: 55.40% Test: 59.20%
Run: 09, Epoch: 200, Loss: 0.2462, Train: 77.50%, Valid: 56.00% Test: 61.30%
Run 09:
Highest Train: 99.17
Highest Valid: 77.60
  Final Train: 81.67
   Final Test: 76.60
Run: 10, Epoch: 01, Loss: 1.9066, Train: 11.67%, Valid: 11.40% Test: 14.80%
Run: 10, Epoch: 02, Loss: 1.8444, Train: 29.17%, Valid: 22.20% Test: 25.30%
Run: 10, Epoch: 03, Loss: 1.6882, Train: 30.83%, Valid: 27.80% Test: 29.40%
Run: 10, Epoch: 04, Loss: 1.5438, Train: 29.17%, Valid: 29.00% Test: 31.80%
Run: 10, Epoch: 05, Loss: 1.5105, Train: 25.00%, Valid: 24.80% Test: 26.00%
Run: 10, Epoch: 06, Loss: 1.4427, Train: 20.83%, Valid: 23.80% Test: 24.70%
Run: 10, Epoch: 07, Loss: 1.4065, Train: 20.00%, Valid: 24.00% Test: 24.90%
Run: 10, Epoch: 08, Loss: 1.3715, Train: 23.33%, Valid: 26.00% Test:

Run: 10, Epoch: 130, Loss: 0.2988, Train: 90.83%, Valid: 74.40% Test: 72.70%
Run: 10, Epoch: 131, Loss: 0.2913, Train: 88.33%, Valid: 73.00% Test: 71.10%
Run: 10, Epoch: 132, Loss: 0.2957, Train: 83.33%, Valid: 74.00% Test: 69.60%
Run: 10, Epoch: 133, Loss: 0.2557, Train: 83.33%, Valid: 73.20% Test: 70.60%
Run: 10, Epoch: 134, Loss: 0.2445, Train: 86.67%, Valid: 73.60% Test: 70.10%
Run: 10, Epoch: 135, Loss: 0.2966, Train: 88.33%, Valid: 70.80% Test: 70.10%
Run: 10, Epoch: 136, Loss: 0.2114, Train: 90.83%, Valid: 70.40% Test: 71.00%
Run: 10, Epoch: 137, Loss: 0.2455, Train: 92.50%, Valid: 70.60% Test: 70.90%
Run: 10, Epoch: 138, Loss: 0.2715, Train: 91.67%, Valid: 71.60% Test: 70.70%
Run: 10, Epoch: 139, Loss: 0.3128, Train: 93.33%, Valid: 70.80% Test: 71.30%
Run: 10, Epoch: 140, Loss: 0.2194, Train: 93.33%, Valid: 70.60% Test: 72.40%
Run: 10, Epoch: 141, Loss: 0.2596, Train: 94.17%, Valid: 72.40% Test: 73.30%
Run: 10, Epoch: 142, Loss: 0.2057, Train: 91.67%, Valid: 72.60% Test: 72.60%

# GCN using CC Fe embaded with MLP using topo fe 

In [7]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')
data = dataset[0]
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [8]:
print(data.edge_index.numpy())

[[ 628  158  486 ... 2820 1643   33]
 [   0    1    1 ... 3324 3325 3326]]


In [9]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [10]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [24]:
print(G.number_of_edges())

9104


In [11]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [12]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [13]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

0  |  48 

2  |  1331 

4  |  795 

6  |  438 

8  |  237 

10  |  139 

12  |  110 

14  |  61 

16  |  33 

18  |  24 

20  |  30 

22  |  19 

24  |  17 

26  |  4 

28  |  3 

30  |  9 

32  |  7 

34  |  4 

36  |  2 

38  |  2 

40  |  1 

42  |  2 

44  |  1 

46  |  2 

54  |  1 

56  |  1 

58  |  1 

60  |  1 

68  |  1 

70  |  1 

102  |  1 

198  |  1 



In [14]:
import pyflagser
Node_fil=[0,2,4,6,8,10,12,14,16,18,20,22,24,30,34]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 3326 (100%)

In [29]:
B0data = pd.DataFrame(topo_betti_0)
B1data = pd.DataFrame(topo_betti_1)
#data.insert(loc=k,column='Class',value=Node_Class)

result = pd.concat([B0data, B1data], axis=1)
result.head()
compression_opts = dict(method='zip',
                        archive_name='Topo_Feature_Citeseer.csv')
result.to_csv('Topo_Feature_Citeseer.zip', index=True,
          compression=compression_opts)

In [30]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [15]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [32]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[3327, 12], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104], topo=[3327, 30])


In [33]:
print(len(topo_fe[0]))

30


In [19]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        #return x.log_softmax(dim=-1)
        return x

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [35]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8085, Train: 16.67%, Valid: 18.80% Test: 16.90%
Run: 01, Epoch: 02, Loss: 1.7911, Train: 16.67%, Valid: 18.80% Test: 16.90%
Run: 01, Epoch: 03, Loss: 1.7503, Train: 16.67%, Valid: 18.80% Test: 16.90%
Run: 01, Epoch: 04, Loss: 1.7251, Train: 16.67%, Valid: 18.80% Test: 16.90%
Run: 01, Epoch: 05, Loss: 1.6989, Train: 16.67%, Valid: 19.20% Test: 17.20%
Run: 01, Epoch: 06, Loss: 1.6610, Train: 30.83%, Valid: 33.80% Test: 31.80%
Run: 01, Epoch: 07, Loss: 1.6202, Train: 33.33%, Valid: 36.20% Test: 36.80%
Run: 01, Epoch: 08, Loss: 1.5999, Train: 47.50%, Valid: 48.40% Test: 49.80%
Run: 01, Epoch: 09, Loss: 1.5437, Train: 50.83%, Valid: 44.40% Test: 48.20%
Run: 01, Epoch: 10, Loss: 1.5431, Train: 50.00%, Valid: 43.40% Test: 46.70%
Run: 01, Epoch: 11, Loss: 1.4953, Train: 48.33%, Valid: 43.20% Test: 45.60%
Run: 01, Epoch: 12, Loss: 1.4425, Train: 38.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 13, Loss: 1.4426, Train: 31.67%, Valid: 32.40% Test: 35.70%
Run: 01, Epo

/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Run: 01, Epoch: 26, Loss: 1.1418, Train: 69.17%, Valid: 69.60% Test: 71.70%
Run: 01, Epoch: 27, Loss: 1.1373, Train: 71.67%, Valid: 66.20% Test: 70.50%
Run: 01, Epoch: 28, Loss: 1.1222, Train: 75.83%, Valid: 66.00% Test: 69.50%
Run: 01, Epoch: 29, Loss: 1.1338, Train: 70.83%, Valid: 63.80% Test: 66.60%
Run: 01, Epoch: 30, Loss: 1.0669, Train: 65.83%, Valid: 61.20% Test: 62.20%
Run: 01, Epoch: 31, Loss: 1.0433, Train: 66.67%, Valid: 61.00% Test: 62.10%
Run: 01, Epoch: 32, Loss: 1.0842, Train: 66.67%, Valid: 60.20% Test: 63.00%
Run: 01, Epoch: 33, Loss: 1.0864, Train: 72.50%, Valid: 63.80% Test: 66.40%
Run: 01, Epoch: 34, Loss: 1.0214, Train: 70.83%, Valid: 65.00% Test: 67.00%
Run: 01, Epoch: 35, Loss: 1.0040, Train: 66.67%, Valid: 63.60% Test: 65.10%
Run: 01, Epoch: 36, Loss: 0.9824, Train: 65.83%, Valid: 60.80% Test: 64.40%
Run: 01, Epoch: 37, Loss: 0.9933, Train: 65.00%, Valid: 62.40% Test: 65.80%
Run: 01, Epoch: 38, Loss: 0.9941, Train: 75.83%, Valid: 65.60% Test: 70.40%
Run: 01, Epo

Run: 01, Epoch: 140, Loss: 0.3954, Train: 73.33%, Valid: 61.20% Test: 63.00%
Run: 01, Epoch: 141, Loss: 0.4598, Train: 80.00%, Valid: 64.40% Test: 65.90%
Run: 01, Epoch: 142, Loss: 0.3661, Train: 86.67%, Valid: 70.00% Test: 70.40%
Run: 01, Epoch: 143, Loss: 0.4143, Train: 93.33%, Valid: 70.80% Test: 71.90%
Run: 01, Epoch: 144, Loss: 0.4123, Train: 95.00%, Valid: 72.60% Test: 72.80%
Run: 01, Epoch: 145, Loss: 0.3825, Train: 88.33%, Valid: 71.60% Test: 73.30%
Run: 01, Epoch: 146, Loss: 0.3942, Train: 88.33%, Valid: 71.20% Test: 72.00%
Run: 01, Epoch: 147, Loss: 0.3850, Train: 88.33%, Valid: 69.00% Test: 68.30%
Run: 01, Epoch: 148, Loss: 0.4146, Train: 85.00%, Valid: 69.00% Test: 67.20%
Run: 01, Epoch: 149, Loss: 0.3792, Train: 85.83%, Valid: 71.20% Test: 69.80%
Run: 01, Epoch: 150, Loss: 0.4423, Train: 83.33%, Valid: 71.80% Test: 68.30%
Run: 01, Epoch: 151, Loss: 0.3490, Train: 82.50%, Valid: 73.80% Test: 72.20%
Run: 01, Epoch: 152, Loss: 0.3521, Train: 81.67%, Valid: 76.80% Test: 74.40%

Run: 02, Epoch: 51, Loss: 0.9450, Train: 79.17%, Valid: 69.40% Test: 67.10%
Run: 02, Epoch: 52, Loss: 0.9651, Train: 79.17%, Valid: 71.20% Test: 68.40%
Run: 02, Epoch: 53, Loss: 0.9260, Train: 77.50%, Valid: 67.00% Test: 65.40%
Run: 02, Epoch: 54, Loss: 0.9189, Train: 76.67%, Valid: 65.20% Test: 63.70%
Run: 02, Epoch: 55, Loss: 0.9061, Train: 75.83%, Valid: 64.80% Test: 65.60%
Run: 02, Epoch: 56, Loss: 0.9108, Train: 75.83%, Valid: 66.20% Test: 66.80%
Run: 02, Epoch: 57, Loss: 0.8414, Train: 75.00%, Valid: 65.80% Test: 66.20%
Run: 02, Epoch: 58, Loss: 0.8429, Train: 70.83%, Valid: 67.20% Test: 66.70%
Run: 02, Epoch: 59, Loss: 0.8735, Train: 68.33%, Valid: 63.20% Test: 63.30%
Run: 02, Epoch: 60, Loss: 0.8696, Train: 68.33%, Valid: 64.60% Test: 64.70%
Run: 02, Epoch: 61, Loss: 0.8125, Train: 74.17%, Valid: 70.60% Test: 69.70%
Run: 02, Epoch: 62, Loss: 0.8275, Train: 74.17%, Valid: 72.40% Test: 72.90%
Run: 02, Epoch: 63, Loss: 0.8115, Train: 72.50%, Valid: 76.60% Test: 71.90%
Run: 02, Epo

...

Run: 08, Epoch: 96, Loss: 0.5795, Train: 81.67%, Valid: 69.00% Test: 69.80%
Run: 08, Epoch: 97, Loss: 0.5704, Train: 82.50%, Valid: 71.20% Test: 72.10%
Run: 08, Epoch: 98, Loss: 0.6551, Train: 90.00%, Valid: 73.60% Test: 76.00%
Run: 08, Epoch: 99, Loss: 0.5885, Train: 92.50%, Valid: 78.40% Test: 77.40%
Run: 08, Epoch: 100, Loss: 0.6015, Train: 90.00%, Valid: 81.20% Test: 77.60%
Run: 08, Epoch: 101, Loss: 0.5482, Train: 87.50%, Valid: 80.20% Test: 77.10%
Run: 08, Epoch: 102, Loss: 0.5193, Train: 85.00%, Valid: 76.80% Test: 75.40%
Run: 08, Epoch: 103, Loss: 0.5215, Train: 85.83%, Valid: 74.80% Test: 73.80%
Run: 08, Epoch: 104, Loss: 0.5680, Train: 89.17%, Valid: 74.00% Test: 74.70%
Run: 08, Epoch: 105, Loss: 0.4974, Train: 90.00%, Valid: 73.40% Test: 74.80%
Run: 08, Epoch: 106, Loss: 0.5851, Train: 90.00%, Valid: 70.60% Test: 72.50%
Run: 08, Epoch: 107, Loss: 0.4956, Train: 88.33%, Valid: 69.60% Test: 70.40%
Run: 08, Epoch: 108, Loss: 0.5471, Train: 87.50%, Valid: 67.40% Test: 68.90%
Run

Run: 09, Epoch: 08, Loss: 1.6082, Train: 29.17%, Valid: 32.60% Test: 34.40%
Run: 09, Epoch: 09, Loss: 1.5697, Train: 30.00%, Valid: 31.60% Test: 35.40%
Run: 09, Epoch: 10, Loss: 1.5462, Train: 28.33%, Valid: 27.80% Test: 29.50%
Run: 09, Epoch: 11, Loss: 1.5046, Train: 30.00%, Valid: 22.80% Test: 26.00%
Run: 09, Epoch: 12, Loss: 1.4849, Train: 35.83%, Valid: 21.20% Test: 25.00%
Run: 09, Epoch: 13, Loss: 1.4649, Train: 36.67%, Valid: 21.60% Test: 24.50%
Run: 09, Epoch: 14, Loss: 1.3978, Train: 40.00%, Valid: 31.20% Test: 32.70%
Run: 09, Epoch: 15, Loss: 1.3991, Train: 50.00%, Valid: 42.80% Test: 42.50%
Run: 09, Epoch: 16, Loss: 1.3661, Train: 60.83%, Valid: 51.40% Test: 48.70%
Run: 09, Epoch: 17, Loss: 1.3546, Train: 58.33%, Valid: 49.00% Test: 47.60%
Run: 09, Epoch: 18, Loss: 1.3534, Train: 45.83%, Valid: 39.20% Test: 37.50%
Run: 09, Epoch: 19, Loss: 1.2995, Train: 25.00%, Valid: 22.20% Test: 25.70%
Run: 09, Epoch: 20, Loss: 1.2843, Train: 22.50%, Valid: 17.20% Test: 20.60%
Run: 09, Epo

Run: 09, Epoch: 121, Loss: 0.4505, Train: 76.67%, Valid: 62.00% Test: 65.50%
Run: 09, Epoch: 122, Loss: 0.4740, Train: 80.83%, Valid: 65.00% Test: 67.70%
Run: 09, Epoch: 123, Loss: 0.4826, Train: 81.67%, Valid: 67.20% Test: 72.40%
Run: 09, Epoch: 124, Loss: 0.4621, Train: 79.17%, Valid: 69.00% Test: 72.70%
Run: 09, Epoch: 125, Loss: 0.4413, Train: 80.00%, Valid: 70.80% Test: 71.40%
Run: 09, Epoch: 126, Loss: 0.4338, Train: 79.17%, Valid: 69.20% Test: 68.20%
Run: 09, Epoch: 127, Loss: 0.4547, Train: 75.83%, Valid: 64.80% Test: 65.30%
Run: 09, Epoch: 128, Loss: 0.4979, Train: 75.00%, Valid: 61.80% Test: 61.90%
Run: 09, Epoch: 129, Loss: 0.4370, Train: 72.50%, Valid: 63.80% Test: 62.80%
Run: 09, Epoch: 130, Loss: 0.4534, Train: 73.33%, Valid: 61.60% Test: 62.90%
Run: 09, Epoch: 131, Loss: 0.5177, Train: 71.67%, Valid: 57.40% Test: 60.30%
Run: 09, Epoch: 132, Loss: 0.3804, Train: 72.50%, Valid: 56.80% Test: 60.80%
Run: 09, Epoch: 133, Loss: 0.4342, Train: 76.67%, Valid: 58.60% Test: 62.90%

Run: 10, Epoch: 34, Loss: 1.1039, Train: 65.83%, Valid: 61.60% Test: 57.50%
Run: 10, Epoch: 35, Loss: 1.1093, Train: 70.83%, Valid: 68.60% Test: 66.80%
Run: 10, Epoch: 36, Loss: 1.0592, Train: 77.50%, Valid: 72.00% Test: 71.90%
Run: 10, Epoch: 37, Loss: 1.1017, Train: 75.83%, Valid: 73.20% Test: 71.90%
Run: 10, Epoch: 38, Loss: 1.0864, Train: 75.00%, Valid: 71.20% Test: 70.30%
Run: 10, Epoch: 39, Loss: 1.0358, Train: 72.50%, Valid: 65.00% Test: 67.30%
Run: 10, Epoch: 40, Loss: 1.0411, Train: 70.00%, Valid: 60.80% Test: 64.30%
Run: 10, Epoch: 41, Loss: 1.0366, Train: 66.67%, Valid: 58.00% Test: 61.50%
Run: 10, Epoch: 42, Loss: 1.0381, Train: 64.17%, Valid: 58.60% Test: 60.70%
Run: 10, Epoch: 43, Loss: 1.0187, Train: 66.67%, Valid: 59.60% Test: 60.40%
Run: 10, Epoch: 44, Loss: 1.0397, Train: 70.00%, Valid: 59.60% Test: 59.40%
Run: 10, Epoch: 45, Loss: 1.0173, Train: 72.50%, Valid: 60.40% Test: 60.20%
Run: 10, Epoch: 46, Loss: 0.9907, Train: 73.33%, Valid: 63.60% Test: 63.10%
Run: 10, Epo

Run: 10, Epoch: 144, Loss: 0.4032, Train: 90.00%, Valid: 68.60% Test: 69.40%
Run: 10, Epoch: 145, Loss: 0.4424, Train: 91.67%, Valid: 69.60% Test: 72.10%
Run: 10, Epoch: 146, Loss: 0.3923, Train: 95.00%, Valid: 74.60% Test: 75.70%
Run: 10, Epoch: 147, Loss: 0.4047, Train: 93.33%, Valid: 76.40% Test: 76.30%
Run: 10, Epoch: 148, Loss: 0.3843, Train: 92.50%, Valid: 77.20% Test: 76.30%
Run: 10, Epoch: 149, Loss: 0.3848, Train: 92.50%, Valid: 78.00% Test: 76.80%
Run: 10, Epoch: 150, Loss: 0.3684, Train: 92.50%, Valid: 78.20% Test: 76.60%
Run: 10, Epoch: 151, Loss: 0.3967, Train: 94.17%, Valid: 78.60% Test: 76.40%
Run: 10, Epoch: 152, Loss: 0.4222, Train: 95.83%, Valid: 78.80% Test: 77.20%
Run: 10, Epoch: 153, Loss: 0.4883, Train: 93.33%, Valid: 74.80% Test: 75.30%
Run: 10, Epoch: 154, Loss: 0.3938, Train: 89.17%, Valid: 73.40% Test: 72.80%
Run: 10, Epoch: 155, Loss: 0.3734, Train: 90.00%, Valid: 69.60% Test: 69.60%
Run: 10, Epoch: 156, Loss: 0.3349, Train: 90.83%, Valid: 67.40% Test: 69.20%

In [20]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104], topo=[3327, 30])


In [22]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8255, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 02, Loss: 1.6687, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 03, Loss: 1.5930, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 04, Loss: 1.5556, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 05, Loss: 1.4838, Train: 28.33%, Valid: 17.60% Test: 18.40%
Run: 01, Epoch: 06, Loss: 1.4461, Train: 60.00%, Valid: 33.80% Test: 29.90%
Run: 01, Epoch: 07, Loss: 1.3829, Train: 67.50%, Valid: 29.00% Test: 24.90%
Run: 01, Epoch: 08, Loss: 1.3312, Train: 78.33%, Valid: 28.60% Test: 24.60%
Run: 01, Epoch: 09, Loss: 1.2807, Train: 79.17%, Valid: 30.00% Test: 26.10%
Run: 01, Epoch: 10, Loss: 1.2469, Train: 81.67%, Valid: 31.80% Test: 27.90%
Run: 01, Epoch: 11, Loss: 1.1909, Train: 83.33%, Valid: 34.60% Test: 31.20%
Run: 01, Epoch: 12, Loss: 1.1732, Train: 91.67%, Valid: 38.60% Test: 35.10%
Run: 01, Epoch: 13, Loss: 1.1409, Train: 96.67%, Valid: 41.40% Test: 38.30%
Run: 01, Epo

Run: 02, Epoch: 06, Loss: 1.4860, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 02, Epoch: 07, Loss: 1.4129, Train: 47.50%, Valid: 19.00% Test: 19.60%
Run: 02, Epoch: 08, Loss: 1.3567, Train: 68.33%, Valid: 24.40% Test: 24.70%
Run: 02, Epoch: 09, Loss: 1.3134, Train: 77.50%, Valid: 36.80% Test: 37.60%
Run: 02, Epoch: 10, Loss: 1.2650, Train: 83.33%, Valid: 35.80% Test: 37.30%
Run: 02, Epoch: 11, Loss: 1.2360, Train: 94.17%, Valid: 35.80% Test: 37.50%
Run: 02, Epoch: 12, Loss: 1.2079, Train: 98.33%, Valid: 39.20% Test: 40.00%
Run: 02, Epoch: 13, Loss: 1.1523, Train: 98.33%, Valid: 43.20% Test: 44.30%
Run: 02, Epoch: 14, Loss: 1.1532, Train: 98.33%, Valid: 46.60% Test: 47.80%
Run: 02, Epoch: 15, Loss: 1.1091, Train: 99.17%, Valid: 49.60% Test: 50.30%
Run: 02, Epoch: 16, Loss: 1.0688, Train: 99.17%, Valid: 51.80% Test: 53.00%
Run: 02, Epoch: 17, Loss: 1.0590, Train: 100.00%, Valid: 53.20% Test: 53.50%
Run: 02, Epoch: 18, Loss: 1.0212, Train: 100.00%, Valid: 54.20% Test: 53.40%
Run: 02, E

Run: 03, Epoch: 12, Loss: 1.1969, Train: 99.17%, Valid: 36.00% Test: 37.70%
Run: 03, Epoch: 13, Loss: 1.1539, Train: 99.17%, Valid: 32.40% Test: 33.50%
Run: 03, Epoch: 14, Loss: 1.1138, Train: 99.17%, Valid: 33.40% Test: 34.00%
Run: 03, Epoch: 15, Loss: 1.1118, Train: 99.17%, Valid: 35.80% Test: 35.30%
Run: 03, Epoch: 16, Loss: 1.0616, Train: 100.00%, Valid: 37.20% Test: 36.60%
Run: 03, Epoch: 17, Loss: 1.0314, Train: 100.00%, Valid: 39.00% Test: 37.20%
Run: 03, Epoch: 18, Loss: 1.0090, Train: 100.00%, Valid: 40.60% Test: 38.10%
Run: 03, Epoch: 19, Loss: 0.9716, Train: 100.00%, Valid: 40.80% Test: 39.30%
Run: 03, Epoch: 20, Loss: 0.9739, Train: 100.00%, Valid: 41.80% Test: 40.50%
Run: 03, Epoch: 21, Loss: 0.9430, Train: 100.00%, Valid: 43.20% Test: 41.80%
Run: 03, Epoch: 22, Loss: 0.9115, Train: 100.00%, Valid: 43.80% Test: 42.60%
Run: 03, Epoch: 23, Loss: 0.8830, Train: 100.00%, Valid: 45.60% Test: 43.00%
Run: 03, Epoch: 24, Loss: 0.8722, Train: 100.00%, Valid: 46.80% Test: 43.70%
Run

Run: 04, Epoch: 18, Loss: 0.9570, Train: 100.00%, Valid: 53.80% Test: 51.90%
Run: 04, Epoch: 19, Loss: 0.9077, Train: 100.00%, Valid: 54.60% Test: 52.70%
Run: 04, Epoch: 20, Loss: 0.8949, Train: 100.00%, Valid: 54.20% Test: 54.20%
Run: 04, Epoch: 21, Loss: 0.8747, Train: 100.00%, Valid: 54.40% Test: 55.00%
Run: 04, Epoch: 22, Loss: 0.8526, Train: 100.00%, Valid: 54.40% Test: 55.60%
Run: 04, Epoch: 23, Loss: 0.8502, Train: 100.00%, Valid: 56.00% Test: 56.50%
Run: 04, Epoch: 24, Loss: 0.8230, Train: 100.00%, Valid: 56.60% Test: 57.10%
Run: 04, Epoch: 25, Loss: 0.8293, Train: 100.00%, Valid: 56.60% Test: 57.10%
Run: 04, Epoch: 26, Loss: 0.7678, Train: 100.00%, Valid: 56.40% Test: 57.50%
Run: 04, Epoch: 27, Loss: 0.7593, Train: 100.00%, Valid: 55.80% Test: 57.60%
Run: 04, Epoch: 28, Loss: 0.7837, Train: 100.00%, Valid: 55.80% Test: 58.20%
Run: 04, Epoch: 29, Loss: 0.7701, Train: 100.00%, Valid: 56.60% Test: 58.40%
Run: 04, Epoch: 30, Loss: 0.7626, Train: 100.00%, Valid: 57.40% Test: 59.10%

Run: 05, Epoch: 24, Loss: 0.8442, Train: 100.00%, Valid: 55.20% Test: 57.90%
Run: 05, Epoch: 25, Loss: 0.8138, Train: 100.00%, Valid: 56.40% Test: 57.70%
Run: 05, Epoch: 26, Loss: 0.8171, Train: 100.00%, Valid: 56.80% Test: 58.10%
Run: 05, Epoch: 27, Loss: 0.8067, Train: 100.00%, Valid: 57.40% Test: 58.40%
Run: 05, Epoch: 28, Loss: 0.7652, Train: 100.00%, Valid: 58.00% Test: 58.70%
Run: 05, Epoch: 29, Loss: 0.7502, Train: 100.00%, Valid: 57.60% Test: 59.10%
Run: 05, Epoch: 30, Loss: 0.7356, Train: 100.00%, Valid: 57.80% Test: 59.30%
Run: 05, Epoch: 31, Loss: 0.7463, Train: 100.00%, Valid: 58.00% Test: 59.40%
Run: 05, Epoch: 32, Loss: 0.7299, Train: 100.00%, Valid: 58.20% Test: 59.50%
Run: 05, Epoch: 33, Loss: 0.7132, Train: 100.00%, Valid: 58.40% Test: 59.40%
Run: 05, Epoch: 34, Loss: 0.7072, Train: 100.00%, Valid: 58.20% Test: 59.80%
Run: 05, Epoch: 35, Loss: 0.6662, Train: 100.00%, Valid: 57.80% Test: 59.90%
Run: 05, Epoch: 36, Loss: 0.6682, Train: 100.00%, Valid: 58.20% Test: 60.60%

Run: 06, Epoch: 30, Loss: 0.8204, Train: 100.00%, Valid: 42.80% Test: 43.60%
Run: 06, Epoch: 31, Loss: 0.7840, Train: 100.00%, Valid: 43.00% Test: 43.60%
Run: 06, Epoch: 32, Loss: 0.7612, Train: 100.00%, Valid: 43.20% Test: 43.60%
Run: 06, Epoch: 33, Loss: 0.7472, Train: 100.00%, Valid: 43.40% Test: 43.60%
Run: 06, Epoch: 34, Loss: 0.7603, Train: 100.00%, Valid: 43.00% Test: 43.50%
Run: 06, Epoch: 35, Loss: 0.7477, Train: 100.00%, Valid: 42.60% Test: 43.50%
Run: 06, Epoch: 36, Loss: 0.6983, Train: 100.00%, Valid: 43.00% Test: 43.30%
Run: 06, Epoch: 37, Loss: 0.6993, Train: 100.00%, Valid: 42.20% Test: 43.30%
Run: 06, Epoch: 38, Loss: 0.6692, Train: 100.00%, Valid: 41.80% Test: 42.80%
Run: 06, Epoch: 39, Loss: 0.6817, Train: 100.00%, Valid: 41.60% Test: 42.70%
Run: 06, Epoch: 40, Loss: 0.6791, Train: 100.00%, Valid: 41.60% Test: 42.80%
Run: 06, Epoch: 41, Loss: 0.6647, Train: 100.00%, Valid: 41.40% Test: 43.10%
Run: 06, Epoch: 42, Loss: 0.6426, Train: 100.00%, Valid: 41.80% Test: 43.00%

Run: 07, Epoch: 36, Loss: 0.7143, Train: 100.00%, Valid: 51.20% Test: 48.50%
Run: 07, Epoch: 37, Loss: 0.6778, Train: 100.00%, Valid: 51.20% Test: 48.70%
Run: 07, Epoch: 38, Loss: 0.6849, Train: 100.00%, Valid: 51.20% Test: 49.00%
Run: 07, Epoch: 39, Loss: 0.6756, Train: 100.00%, Valid: 51.40% Test: 49.30%
Run: 07, Epoch: 40, Loss: 0.6395, Train: 100.00%, Valid: 51.20% Test: 49.70%
Run: 07, Epoch: 41, Loss: 0.6677, Train: 100.00%, Valid: 51.40% Test: 49.90%
Run: 07, Epoch: 42, Loss: 0.6422, Train: 100.00%, Valid: 51.20% Test: 50.10%
Run: 07, Epoch: 43, Loss: 0.6332, Train: 100.00%, Valid: 50.60% Test: 49.60%
Run: 07, Epoch: 44, Loss: 0.6087, Train: 100.00%, Valid: 50.60% Test: 49.20%
Run: 07, Epoch: 45, Loss: 0.6431, Train: 100.00%, Valid: 50.80% Test: 49.20%
Run: 07, Epoch: 46, Loss: 0.6220, Train: 100.00%, Valid: 50.80% Test: 49.20%
Run: 07, Epoch: 47, Loss: 0.6127, Train: 100.00%, Valid: 50.60% Test: 48.90%
Run: 07, Epoch: 48, Loss: 0.5645, Train: 100.00%, Valid: 50.60% Test: 49.10%

Run: 08, Epoch: 42, Loss: 0.6397, Train: 100.00%, Valid: 51.80% Test: 51.70%
Run: 08, Epoch: 43, Loss: 0.6321, Train: 100.00%, Valid: 52.20% Test: 52.00%
Run: 08, Epoch: 44, Loss: 0.6321, Train: 100.00%, Valid: 52.60% Test: 52.10%
Run: 08, Epoch: 45, Loss: 0.6257, Train: 100.00%, Valid: 52.40% Test: 52.40%
Run: 08, Epoch: 46, Loss: 0.6155, Train: 100.00%, Valid: 52.00% Test: 52.40%
Run: 08, Epoch: 47, Loss: 0.5876, Train: 100.00%, Valid: 51.40% Test: 52.00%
Run: 08, Epoch: 48, Loss: 0.5863, Train: 100.00%, Valid: 51.00% Test: 51.80%
Run: 08, Epoch: 49, Loss: 0.5778, Train: 100.00%, Valid: 50.40% Test: 51.30%
Run: 08, Epoch: 50, Loss: 0.5582, Train: 100.00%, Valid: 50.20% Test: 50.50%
Run: 08, Epoch: 51, Loss: 0.5733, Train: 100.00%, Valid: 50.20% Test: 50.50%
Run: 08, Epoch: 52, Loss: 0.5603, Train: 100.00%, Valid: 49.40% Test: 50.10%
Run: 08, Epoch: 53, Loss: 0.5418, Train: 100.00%, Valid: 50.00% Test: 50.10%
Run: 08, Epoch: 54, Loss: 0.5398, Train: 100.00%, Valid: 50.20% Test: 50.60%

Run: 09, Epoch: 48, Loss: 0.6327, Train: 100.00%, Valid: 59.00% Test: 55.10%
Run: 09, Epoch: 49, Loss: 0.6076, Train: 100.00%, Valid: 59.40% Test: 55.50%
Run: 09, Epoch: 50, Loss: 0.5857, Train: 100.00%, Valid: 59.40% Test: 55.70%
Run: 09, Epoch: 51, Loss: 0.5887, Train: 100.00%, Valid: 59.00% Test: 55.80%
Run: 09, Epoch: 52, Loss: 0.5730, Train: 100.00%, Valid: 59.20% Test: 55.70%
Run: 09, Epoch: 53, Loss: 0.5748, Train: 100.00%, Valid: 59.00% Test: 55.50%
Run: 09, Epoch: 54, Loss: 0.5935, Train: 100.00%, Valid: 59.00% Test: 55.40%
Run: 09, Epoch: 55, Loss: 0.5504, Train: 100.00%, Valid: 59.20% Test: 55.30%
Run: 09, Epoch: 56, Loss: 0.5568, Train: 100.00%, Valid: 58.60% Test: 54.90%
Run: 09, Epoch: 57, Loss: 0.5602, Train: 100.00%, Valid: 58.80% Test: 54.90%
Run: 09, Epoch: 58, Loss: 0.5085, Train: 100.00%, Valid: 59.40% Test: 55.20%
Run: 09, Epoch: 59, Loss: 0.5029, Train: 100.00%, Valid: 59.80% Test: 55.40%
Run: 09, Epoch: 60, Loss: 0.4974, Train: 100.00%, Valid: 60.80% Test: 55.60%

Run: 10, Epoch: 54, Loss: 0.5291, Train: 100.00%, Valid: 57.20% Test: 57.50%
Run: 10, Epoch: 55, Loss: 0.5336, Train: 100.00%, Valid: 57.40% Test: 57.40%
Run: 10, Epoch: 56, Loss: 0.5601, Train: 100.00%, Valid: 57.60% Test: 57.70%
Run: 10, Epoch: 57, Loss: 0.5112, Train: 100.00%, Valid: 57.40% Test: 57.80%
Run: 10, Epoch: 58, Loss: 0.5129, Train: 100.00%, Valid: 57.60% Test: 57.60%
Run: 10, Epoch: 59, Loss: 0.5081, Train: 100.00%, Valid: 57.40% Test: 57.30%
Run: 10, Epoch: 60, Loss: 0.5034, Train: 100.00%, Valid: 56.60% Test: 57.40%
Run: 10, Epoch: 61, Loss: 0.4740, Train: 100.00%, Valid: 56.20% Test: 57.50%
Run: 10, Epoch: 62, Loss: 0.4776, Train: 100.00%, Valid: 56.40% Test: 57.90%
Run: 10, Epoch: 63, Loss: 0.4706, Train: 100.00%, Valid: 57.00% Test: 57.90%
Run: 10, Epoch: 64, Loss: 0.4760, Train: 100.00%, Valid: 57.20% Test: 57.80%
Run: 10, Epoch: 65, Loss: 0.4477, Train: 100.00%, Valid: 57.80% Test: 57.70%
Run: 10, Epoch: 66, Loss: 0.4584, Train: 100.00%, Valid: 57.60% Test: 57.70%

In [ ]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()
print(data)
train_idx = torch.tensor(np.where(data.train_mask)[0])
valid_idx = np.where(data.val_mask)[0]
test_idx = np.where(data.test_mask)[0]
print(train_idx)

print(len(data.y.squeeze()[train_idx]))

In [ ]:
print(data.adj_t)

In [ ]:
array=np.array([[1,2],[2,3]])
data.x=torch.tensor(array)

In [ ]:
print(data)

In [ ]:
array2=np.array([[1,2],[2,3]])
array1=np.array([[1,-2],[3,4]])
array2=torch.tensor(array2)
array1=torch.tensor(array1)
ConArray=torch.cat((array2, array1), 1)
print(ConArray)

In [ ]:
# input is of size N x C = 3 x 5
input = torch.randn(3, 6, requires_grad=True)
print(input)


In [ ]:
inp=torch.tensor([[ 1.2646, -1.1952,  1.0827,  0.8987, -0.0378],
        [ 0.5404,  1.3919, -0.1529, -1.9264, -0.9797],
        [ 0.4057,  2.5791,  0.4627,  0.6131, -1.5965]])
inp1=torch.tensor([[ 1.2646, -1.1952,  1.0827,  0.8987, -0.0378, -1.7763],
        [ 0.5404,  1.3919, -0.1529, -1.9264, -0.9797,  0.0765],
        [ 0.4057,  2.5791,  0.4627,  0.6131, -1.5965, -0.8306]])
target = torch.tensor([2, 4, 0])
print(target)
output = F.nll_loss(F.log_softmax(inp, dim=1), target)
output1 = F.nll_loss(F.log_softmax(inp1, dim=1), target)
print(output)
print(output1)

In [ ]:
# each element in target has to have 0 <= value < C
target = torch.tensor([2, 4, 0])
print(target)
output = F.nll_loss(F.log_softmax(input, dim=1), target)
print(output)
output.backward()